In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

## Image resampling

Using simple morphological operations, an X-ray image with some set of intrinsic parameters can be resampled with a different set of intrinsics.

In [ ]:
#| export
import torch
from kornia.geometry.transform import center_crop, resize, translate


def resample(
    img: torch.Tensor,
    focal_len: float,
    delx: float,
    x0: float = 0,
    y0: float = 0,
    new_focal_len: float = None,
    new_delx: float = None,
    new_x0: float = None,
    new_y0: float = None,
) -> torch.Tensor:
    """Resample an image with new intrinsic parameters."""
    if new_focal_len is None:
        new_focal_len = focal_len
    if new_delx is None:
        new_delx = delx
    if new_x0 is None:
        new_x0 = x0
    if new_y0 is None:
        new_y0 = y0

    x = img.clone()
    _, _, height, width = x.shape
    shape = torch.tensor([height, width])

    # Translate the image
    translation = torch.tensor([[new_x0 - x0, new_y0 - y0]]) / delx
    x = translate(x, translation.to(x))

    # Crop the image to change the focal length
    focal_scaling = new_focal_len / focal_len
    crop_size = (shape / focal_scaling).to(int).tolist()
    x = center_crop(x, crop_size)
    x = resize(x, (height, width))

    # Pad the image to resize pixels
    pixel_scaling = new_delx / delx
    padding = (shape * (pixel_scaling - 1) / 2).to(int).tolist()
    padding = [padding[1], padding[1], padding[0], padding[0]]
    x = torch.nn.functional.pad(x, padding)
    x = resize(x, (height, width))

    return x

## Pinhole camera's

Convert the intrinsic and extrinsic geometries, as implemented in DiffDRR, to a standard pinhole camera.

In [ ]:
#| exporti
from kornia.geometry.camera.pinhole import PinholeCamera as KorniaPinholeCamera
from torchio import Subject

from diffdrr.detector import Detector


class PinholeCamera(KorniaPinholeCamera):
    def __init__(
        self,
        intrinsics: torch.Tensor,
        extrinsics: torch.Tensor,
        height: torch.Tensor,
        width: torch.Tensor,
        detector: Detector,
        subject: Subject,
        
    ):
        super().__init__(intrinsics, extrinsics, height, width)
        multiplier = -1 if subject.orientation == "PA" else 1
        self.sdd = multiplier * detector.sdd
        self.delx = detector.delx
        self.dely = detector.dely
        self.x0 = detector.x0
        self.y0 = detector.y0

    @property
    def center(self):
        """Camera center is -R^T t (Hartley and Zisserman)"""
        return (-self.extrinsics[:, :3, :3].mT @ self.extrinsics[:, :3, 3:])[..., 0]

    @property
    def projmat(self):
        """Bx3x4 camera matrix"""
        return (self.intrinsics @ self.extrinsics)[:, :3]

    @property
    def pose(self):
        """Turn c2w matrix in w2c matrix as input to diffdrr.drr.DRR"""
        return RigidTransform(self.extrinsics).inverse()

In [ ]:
#| export
from copy import deepcopy

from kornia.geometry.calibration import solve_pnp_dlt

from diffdrr.detector import make_intrinsic_matrix
from diffdrr.drr import DRR
from diffdrr.pose import RigidTransform


def get_pinhole_camera(
    drr: DRR, pose: RigidTransform, dtype: torch.dtype = torch.float64
) -> PinholeCamera:
    # Move everything to CPU and use double precision
    drr = deepcopy(drr).to(device="cpu", dtype=dtype)
    pose = deepcopy(pose).to(device="cpu", dtype=dtype)

    # Make the intrinsic matrix (in pixels)
    multiplier = -1 if drr.subject.orientation == "AP" else 1
    fx = multiplier * drr.detector.sdd / drr.detector.delx
    fy = multiplier * drr.detector.sdd / drr.detector.dely
    u0 = drr.detector.x0 / drr.detector.delx + drr.detector.width / 2
    v0 = drr.detector.y0 / drr.detector.dely + drr.detector.height / 2
    intrinsics = torch.tensor(
        [
            [
                [fx, 0.0, u0, 0.0],
                [0.0, fy, v0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 0.0, 1.0],
            ]
        ],
        dtype=dtype,
    )

    # Get matching 3D and 2D points for PnP
    (xmin, xmax), (ymin, ymax), (zmin, zmax) = drr.subject.volume.get_bounds()
    X = torch.tensor(
        [
            [
                [xmin, ymin, zmin],
                [xmax, ymin, zmin],
                [xmin, ymax, zmin],
                [xmax, ymax, zmin],
                [xmin, ymin, zmax],
                [xmax, ymin, zmax],
                [xmin, ymax, zmax],
                [xmax, ymax, zmax],
            ]
        ],
        dtype=dtype,
    )
    x = drr.perspective_projection(pose, X)

    # Solve for the extrinsic matrix with PnP
    extrinsics = torch.eye(4, dtype=dtype)[None]
    extrinsics[:, :3, :] = solve_pnp_dlt(X, x, intrinsics[..., :3, :3])

    # Make the pinhole camera, converted back to single precision
    camera = PinholeCamera(
        intrinsics.to(torch.float32),
        extrinsics.to(torch.float32),
        torch.tensor([drr.detector.height]),
        torch.tensor([drr.detector.width]),
        drr.detector,
        drr.subject,
    )

    return camera

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()